# Lambda 1 - Getting Tweets which contain #StandWithUkraine

In [ ]:
#make sure to install the Lambda layer beforehand
import pandas as pd
import tweepy
import boto3
from io import StringIO
import time



def lambda_handler(event, context):
    
    # call s3 bucket
    s3 = boto3.resource('s3')
    
    #api keys
    consumer_key = "your_consumer_key"
    consumer_secret= "your_consumer_secret"
    access_token= "your_access_token"
    access_token_secret= "your_access_token_secret"
    
    #api auth
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth)
    auth.set_access_token(access_token, access_token_secret)
    
    #twitter api parameters
    number_of_tweets = 100
    tweets_ukraine = []
    columns = ['Time Created', 'Number_of_Likes', 'Number_of_Retweets', 'Tweet']
    search_words = '#StandWithUkraine'
    lang = 'en'
    
    #getting the data
    for tweet in tweepy.Cursor(api.search_tweets, q=search_words, lang = lang, 
                        tweet_mode='extended').items(number_of_tweets):
                            tweets_ukraine.append([tweet.created_at, tweet.favorite_count, tweet.retweet_count, tweet.full_text])
    #create a dataframe
    df = pd.DataFrame(tweets_ukraine, columns=columns)
    
    #define filename
    csv_time = time.strftime("%Y%m%d%H%M")
    csv_name = "twitterapi" + csv_time + '.csv'

    # name of existing bucket
    bucket = 'your_bucket_name' 
    
    #save it into a buffer
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    s3.Object(bucket, csv_name).put(Body=csv_buffer.getvalue())
